[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1t0_4BxEJ0XncyYvn_VyEQhxwNMvtSUNx?usp=sharing)

In [1]:
import sys
sys.path.append('../src')

import torch
import torch.nn as nn
from copy import deepcopy
from pathlib import Path
from dgib import DGIB, ExtractorMLP, Discriminator
from utils import get_data_loaders, get_model, set_seed, CriterionClf, CriterionRecon, init_metric_dict, load_checkpoint
from trainer import run_one_epoch, update_best_epoch_res, get_viz_idx, visualize_results
from datetime import datetime


ModuleNotFoundError: No module named 'dgib'

In [ ]:
#dataset_name = 'ba_2motifs'
# dataset_name = 'mutag'
# model_name = 'GIN'

dataset_name = 'ImageMagick'
model_name = 'PNA'

method_name = 'DGIB'
cuda_id = 0
seed = 0
set_seed(seed)

In [ ]:
data_dir = Path('../data')
device = torch.device(f'cuda:{cuda_id}' if cuda_id >= 0 else 'cpu')

if model_name == 'GIN':
    model_config = {'model_name': 'GIN', 'hidden_size': 80, 'n_layers': 2, 'dropout_p': 0.3, 'use_edge_attr': True}
elif model_name == 'PNA':
    model_config = {'model_name': 'PNA', 'hidden_size': 80, 'n_layers': 4, 'dropout_p': 0.3, 'use_edge_attr': False, 
                    'atom_encoder': False, 'aggregators': ['mean', 'min', 'max', 'std'], 'scalers': False}
else:
    assert model_name == 'RGCN'
    model_config = {'model_name': 'RGCN', 'hidden_size': 64, 'n_layers': 2, 'dropout_p': 0.3, 'use_edge_attr': False}
metric_dict = deepcopy(init_metric_dict)
model_dir = data_dir / dataset_name / 'logs' / (datetime.now().strftime("%m_%d_%Y-%H_%M_%S") + '-' + dataset_name + '-' + model_name + '-seed' + str(seed) + '-' + method_name)


In [ ]:
loaders, test_set, x_dim, edge_attr_dim, num_class, aux_info = get_data_loaders(data_dir, dataset_name, batch_size=128, random_state=seed,
                                                                                splits={'train': 0.8, 'valid': 0.1, 'test': 0.1}, mode='common', parameters=None,
                                                                                mutag_x=True if dataset_name == 'mutag' else False)
model_config['deg'] = aux_info['deg']

In [ ]:
# print(test_set[0].num_relations)
print(aux_info)

In [ ]:
gnn_t = get_model(x_dim, edge_attr_dim, num_class, aux_info['multi_label'], model_config, device, aux_info.get('num_relations', None))
gnn_s = get_model(x_dim, edge_attr_dim, num_class, aux_info['multi_label'], model_config, device, aux_info.get('num_relations', None))
extractor = ExtractorMLP(model_config['hidden_size'], learn_edge_att=False).to(device)
discriminator = Discriminator(model_config['hidden_size']).to(device)
optimizer = torch.optim.Adam(list(extractor.parameters()) + list(gnn_t.parameters()) + list(gnn_s.parameters()) + list(discriminator.parameters()), lr=1e-3, weight_decay=3.0e-6)
criterion_clf = CriterionClf(num_class, aux_info['multi_label'])
criterion_recon = CriterionRecon()
dgib = DGIB(gnn_t, gnn_s, extractor, discriminator, criterion_clf, criterion_recon, optimizer, learn_edge_att=False, final_r=0.7)

In [ ]:

print(loaders['train'].__dict__)
for batch in loaders['train']:
    print(batch)
    """ print(batch.edge_index)
    print(batch.batch) """
    """ print(len(batch))
    print(batch)
print(aux_info['multi_label'])
print(criterion_clf.multi_label) """

In [ ]:
""" import numpy as np
array1=batch.edge_index.numpy()
array2=batch.batch.numpy()
with open('tensors.txt', 'w') as f:
    # 将第一个tensor保存到文件
    np.savetxt(f, array1, fmt='%d', header='Tensor 1', comments='')
    
    # 添加一个空行作为分隔符
    f.write('\n')
    
    # 将第二个tensor保存到文件
    np.savetxt(f, array2, fmt='%d', header='Tensor 2', comments='') """

In [ ]:
# torch.autograd.set_detect_anomaly(True)
for epoch in range(200):
    train_res = run_one_epoch(dgib, loaders['train'], epoch, 'train', dataset_name, seed, model_config['use_edge_attr'], aux_info['multi_label'])
    valid_res = run_one_epoch(dgib, loaders['valid'], epoch, 'valid', dataset_name, seed, model_config['use_edge_attr'], aux_info['multi_label'])
    test_res = run_one_epoch(dgib, loaders['test'], epoch, 'test', dataset_name, seed, model_config['use_edge_attr'], aux_info['multi_label'])
    
    metric_dict = update_best_epoch_res(dgib, train_res, valid_res, test_res, metric_dict, dataset_name, epoch, model_dir)
    print(f'[Seed {seed}, Epoch: {epoch}]: Best Epoch: {metric_dict["metric/best_clf_epoch"]}, '
          f'Best Val Pred ACC/ROC: {metric_dict["metric/best_clf_valid"]:.3f}, Best Test Pred ACC/ROC: {metric_dict["metric/best_clf_test"]:.3f}, '
          f'Best Test X AUROC: {metric_dict["metric/best_x_roc_test"]:.3f}')
    print('='*50)
    print('='*50)

In [ ]:
best_epoch = metric_dict['metric/best_clf_epoch']
load_checkpoint(dgib, model_dir, model_name=f'dgib_epoch_{best_epoch}', map_location=device)

In [ ]:
num_viz_samples = 10
assert aux_info['multi_label'] is False

all_viz_set = get_viz_idx(test_set, dataset_name, num_viz_samples)
visualize_results(dgib, all_viz_set, test_set, num_viz_samples, dataset_name, model_config['use_edge_attr'])

In [ ]:
print(all_viz_set)